<a href="https://colab.research.google.com/github/pkse-searcher/fsvm-pois-attack-defense/blob/main/Attack_Project_Progress_11_6_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import svm
from random import randint, shuffle, uniform
from copy import copy,deepcopy
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import itertools

from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from math import isclose, fabs

from joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import pip

def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])

try:
  import secml
except:
  install('git+https://gitlab.com/secml/secml')
  import secml
from secml.ml.classifiers import CClassifierSVM
from secml.ml.kernels import CKernelRBF
from secml.adv.attacks import CAttackPoisoningSVM
from secml.data import CDataset
from secml.data.splitter import CTrainTestSplit
from secml.ml.kernels.c_kernel_linear import CKernelLinear
from secml.data.loader import CDLRandomBlobs
from secml.array import CArray
from sklearn.model_selection import train_test_split
# NBVAL_IGNORE_OUTPUT
from secml.data.loader import CDataLoaderMNIST, CDataLoaderCIFAR10


import gzip
import struct
from array import array
from multiprocessing import Lock
import numpy as np

from secml.data.loader import CDataLoader
from secml.data import CDataset, CDatasetHeader
from secml.array import CArray
from secml.utils import fm
from secml.utils.download_utils import dl_file_gitlab, md5
from secml.settings import SECML_DS_DIR




try:
  from pyod.models.knn import KNN
except:
  install('pyod')
  from pyod.models.knn import KNN
from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.copod import COPOD
import matplotlib.pyplot as plt

In [ ]:
def model_change(advNumber, local_machine, w, b):
    if local_machine>=advNumber: # if non attacker no model is changed
        return w, b
    global old_w, old_b
    if MODE == 'COLLABORAT' and local_machine != 0:
        return old_w, old_b
    if ATTACK_TYPE == 'RANDOM':
        tw = deepcopy(w)
        mx = w.max()
        mn = w.min()
        for x in range(w.size):
            tw[x] = uniform(mn, mx)
        old_w, old_b = tw, b
        return tw, b



    if ATTACK_TYPE == 'AGR_KRUM':
        tw = deepcopy(w)
        all_updates = []
        for adv_local_machine in range(advNumber):
            adv_Xlocal = deepcopy(XGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])
            adv_ylocal = deepcopy(yGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])

            # print(local_machine, y_local)

            adv_clfLocals = CClassifierSVM(C=1)
            adv_clfLocals.fit(adv_Xlocal, adv_ylocal)

            adv_w, adv_b = adv_clfLocals.w.get_data()[0], adv_clfLocals.b.get_data()[0]
            adv_m = np.sqrt(adv_w.dot(adv_w))
            adv_w = adv_w / adv_m
            adv_b /= adv_m
            all_updates.append(np.append(adv_w,adv_b))

        model_re = np.append(w,b)
        n_attackers = advNumber
        dev_type = 'std'
        deviation = np.std(all_updates, 0)

        lamda = 3.0

        threshold_diff = 1e-5
        lamda_fail = lamda
        lamda_succ = 0

        while fabs(lamda_succ - lamda) > threshold_diff:
            mal_update = (model_re - lamda * deviation)
            # print(mal_update)
            mal_updates = np.full((n_attackers, mal_update.size), mal_update)
            # print(f'{mal_updates[0].shape} and {all_updates[0].shape}')
            # print(f'{len(mal_updates)} and {len(all_updates)}')
            mal_updates = np.concatenate((mal_updates, all_updates))

            hSets =  mal_updates
            krum_scores = []
            byzantine_client_num = len(hSets) // 5
            # print(f'lambda {lambda_} number of adversary {advNumber} and updates {len(all_updates)} and hSets {len(hSets)}')
            for i in range(0, len(hSets)):
                dists = []
                for j in range(0, len(hSets)):
                    if i != j:
                        temp_vector = np.subtract(hSets[i], hSets[j])
                        distance = np.dot(temp_vector,temp_vector)
                        dists.append(distance)
                #   print(i, "all_socre", dists)
                dists.sort()  # ascending
                score = dists[0 : len(hSets) - byzantine_client_num - 2]
                # print(i, "low_socre", score)
                krum_scores.append(sum(score))
                # print(i, krum_scores[-1])
            score_index = np.argsort(krum_scores)  # indices; ascending
            #   print(krum_scores)
            #   print(score_index)
            krum_candidate = score_index[0 : len(hSets) - byzantine_client_num - 2]

            if np.sum(krum_candidate < n_attackers) == n_attackers:
                # print('successful lamda is ', lamda)
                lamda_succ = lamda
                lamda = lamda + lamda_fail / 2
            else:
                lamda = lamda - lamda_fail / 2

            lamda_fail = lamda_fail / 2

        mal_update = (model_re - lamda_succ * deviation)
        tw = mal_update[0: len(mal_update) - 1]
        b = mal_update[len(mal_update) - 1]
        # print(f'The lambda = {lambda_}\n tw = {tw}\n b = {b}\n')
        old_w, old_b = tw,b
        return tw, b




    if ATTACK_TYPE == 'LIE':
        tw = deepcopy(w)
        num_std = 1.5
        users_grads = []
        for adv_local_machine in range(advNumber):
            adv_Xlocal = deepcopy(XGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])
            adv_ylocal = deepcopy(yGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])

            # print(local_machine, y_local)

            adv_clfLocals = CClassifierSVM(C=1)
            adv_clfLocals.fit(adv_Xlocal, adv_ylocal)

            adv_w, adv_b = adv_clfLocals.w.get_data()[0], adv_clfLocals.b.get_data()[0]
            adv_m = np.sqrt(adv_w.dot(adv_w))
            adv_w = adv_w / adv_m
            adv_b /= adv_m
            users_grads.append(np.append(adv_w,adv_b))

        grads_mean = np.mean(users_grads, axis=0)
        grads_stdev = np.var(users_grads, axis=0) ** 0.5

        grads_mean[:] -= num_std * grads_stdev[:]

        tw = grads_mean[0: -1]
        b = grads_mean[ -1]
        print(f'The grads_mean = {grads_mean.shape}\n tw = {tw.shape}\n b = {b}\n')
        old_w, old_b = tw,b
        return tw, b

    if ATTACK_TYPE == 'FANG_KRUM':
        tw = deepcopy(w)

        agg_updates = np.append(tw,b)
        deviation = np.sign(agg_updates)

        all_updates = []
        for adv_local_machine in range(advNumber):
            adv_Xlocal = deepcopy(XGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])
            adv_ylocal = deepcopy(yGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])

            # print(local_machine, y_local)

            adv_clfLocals = CClassifierSVM(C=1)
            adv_clfLocals.fit(adv_Xlocal, adv_ylocal)

            adv_w, adv_b = adv_clfLocals.w.get_data()[0], adv_clfLocals.b.get_data()[0]
            adv_m = np.sqrt(adv_w.dot(adv_w))
            adv_w = adv_w / adv_m
            adv_b /= adv_m
            all_updates.append(np.append(adv_w,adv_b))

        # lambda_ = 5
        st, fn = 1e-5, 5
        while fabs(fn-st) > 1e-5 and st<fn:
            lambda_ = (st+fn) / 2
            adv_updates = agg_updates - lambda_ * deviation
            # print(f'adv_updates {adv_updates}')
            hSets = np.vstack([all_updates, adv_updates])

            krum_scores = []
            byzantine_client_num = 1
            # print(f'lambda {lambda_} number of adversary {advNumber} and updates {len(all_updates)} and hSets {len(hSets)}')
            for i in range(0, len(hSets)):
                dists = []
                for j in range(0, len(hSets)):
                    if i != j:
                        temp_vector = np.subtract(hSets[i], hSets[j])
                        distance = np.dot(temp_vector,temp_vector)
                        dists.append(distance)
                #   print(i, "all_socre", dists)
                dists.sort()  # ascending
                score = dists[0 : len(hSets) - byzantine_client_num - 2]
                # print(i, "low_socre", score)
                krum_scores.append(sum(score))
                # print(i, krum_scores[-1])
            score_index = np.argsort(krum_scores)  # indices; ascending
            #   print(krum_scores)
            #   print(score_index)
            score_index = score_index[0 : len(hSets) - byzantine_client_num - 2]
            if advNumber in score_index:
                st = lambda_
            else:
                fn = lambda_
        tw = adv_updates[0: len(adv_updates) - 1]
        b = adv_updates[len(adv_updates) - 1]
        # print(f'The lambda = {lambda_}\n tw = {tw}\n b = {b}\n')
        old_w, old_b = tw,b
        return tw, b

    if ATTACK_TYPE == 'OUR_ATTACK_BND_KRUM':
        tw = deepcopy(w)

        agg_updates = np.append(tw,b)
        deviation = np.sign(agg_updates)

        for j in range(w.size):
            sum_ = 0
            for i in range(len(XGlobal)):
                sum_ += (-1 if yGlobal[i] == 0 else 1) * XGlobal[i][j]
            print(f'j = {j} deviation {deviation[j]} sum = {sum_} new {1 if sum_ >= 0 else -1}')
            agg_updates[j] = -fabs(w[j]) if sum_ >= 0 else fabs(w[j])
            deviation[j] = -1 if sum_ >= 0 else 1
            # tw[j] = -1000 if sum_ >= 0 else 1000
            # print(j, w[j], tw[j], sum_)


        all_updates = []
        for adv_local_machine in range(advNumber):
            adv_Xlocal = deepcopy(XGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])
            adv_ylocal = deepcopy(yGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])

            # print(local_machine, y_local)

            adv_clfLocals = CClassifierSVM(C=1)
            adv_clfLocals.fit(adv_Xlocal, adv_ylocal)

            adv_w, adv_b = adv_clfLocals.w.get_data()[0], adv_clfLocals.b.get_data()[0]
            adv_m = np.sqrt(adv_w.dot(adv_w))
            adv_w = adv_w / adv_m
            adv_b /= adv_m
            all_updates.append(np.append(adv_w,adv_b))

        # lambda_ = 5
        st, fn = 1e-5, 20
        while fabs(fn-st) > 1e-5 and st<fn:
            lambda_ = (st+fn) / 2
            adv_updates = agg_updates - lambda_ * deviation
            # print(f'adv_updates {adv_updates}')
            hSets = np.vstack([all_updates, adv_updates])

            krum_scores = []
            byzantine_client_num = 1
            print(f'lambda {lambda_} number of adversary {advNumber} and updates {len(all_updates)} and hSets {len(hSets)}')
            for i in range(0, len(hSets)):
                dists = []
                for j in range(0, len(hSets)):
                    if i != j:
                        temp_vector = np.subtract(hSets[i], hSets[j])
                        distance = np.dot(temp_vector,temp_vector)
                        dists.append(distance)
                #   print(i, "all_socre", dists)
                dists.sort()  # ascending
                score = dists[0 : len(hSets) - byzantine_client_num - 2]
                # print(i, "low_socre", score)
                krum_scores.append(sum(score))
                # print(i, krum_scores[-1])
            score_index = np.argsort(krum_scores)  # indices; ascending
            #   print(krum_scores)
            #   print(score_index)
            score_index = score_index[0 : len(hSets) - byzantine_client_num - 2]
            if advNumber == score_index[0]:
                st = lambda_
            else:
                fn = lambda_
        tw = adv_updates[0: len(adv_updates) - 1]
        b = adv_updates[len(adv_updates) - 1]
        print(f'The lambda = {lambda_}\n tw = {tw}\n b = {b}\n')
        old_w, old_b = tw,b
        return tw, b

    if ATTACK_TYPE == 'OUR_ATTACK':
        tw = deepcopy(w)
        mx = w.max()
        mn = w.min()
        # print(mn, mx)
        for j in range(w.size):
            sum_ = 0
            for i in range(len(XGlobal)):
                sum_ += (-1 if yGlobal[i] == 0 else 1) * XGlobal[i][j]
                # print('\t', i, j, -1 if testy[i] == 0 else 1, testX[i][j], sum_)
            # tw[j] = mn if sum_ >= 0 else mx
            tw[j] = -fabs(w[j]) if sum_ >= 0 else fabs(w[j])
            # tw[j] = -1000 if sum_ >= 0 else 1000
            # print(j, w[j], tw[j], sum_)
        m = np.sqrt(tw.dot(tw))
        # print(m)
        # tw = tw / m
        old_w, old_b = tw,b
        return tw, b

    if ATTACK_TYPE == 'OUR_ATTACK_GD':
        h=1e-5
        tw = deepcopy(w)
        def get_error(tw, debug = False):
            w_g = (advNumber*tw + (num_local_machine-advNumber)*w)/num_local_machine
            # temp_vector = np.subtract(tw, w)
            # distance = np.dot(temp_vector,temp_vector)
            error_sum=0
            for i in range(len(testX)):
                res = 1+(-1 if testy[i] == 0 else 1) * (np.dot(testX[i],w_g)+b)
                # print(f"res {res} y {yGlobal[i]} and det {np.dot(testX[i],w_g)+b}")
                error_sum += max(0, res)
            # if debug: print(f'iter {_} i {i} distance {distance} error_sum {error_sum}')
            return error_sum # + 500*distance
        iter=40
        prev_acc = 0
        mx = w.max()
        mn = w.min()
        for _ in range(iter):
            cur_acc = get_error(tw, True)
            if(cur_acc < 1e-3 or fabs(prev_acc-cur_acc)/cur_acc < 1e-5):
                break
            cnt = 0
            for i in range(len(testX)):
                det = np.dot(np.append(testX[i], 1), np.append(tw,b))
                if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
            # print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after iter {_} and error {cur_acc} and ratio with previous {fabs(prev_acc-cur_acc)/cur_acc}')
            prev_acc = cur_acc


            for j in range(w.size):
                tw[j]=tw[j]+h
                err_add = get_error(tw)

                tw[j]=tw[j]-2*h
                err_sub = get_error(tw)

                tw[j]=tw[j]+h
                gradient = (err_add-err_sub)/(2*h)
                # print(err_add,err_sub, gradient)

                tw[j]=tw[j]-0.03*gradient
                # if tw[j] > 2*mx: tw[j] = 2*mx
                # if tw[j] < 2*mn: tw[j] = 2*mn

        # agg_updates = np.append(tw,b)
        # orig_updates = np.append(w,b)

        # all_updates = []
        # for adv_local_machine in range(advNumber):
        #     adv_Xlocal = deepcopy(XGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])
        #     adv_ylocal = deepcopy(yGlobal[adv_local_machine * slc : (adv_local_machine + 1) * slc])

        #     # print(local_machine, y_local)

        #     adv_clfLocals = CClassifierSVM(C=1)
        #     adv_clfLocals.fit(adv_Xlocal, adv_ylocal)

        #     adv_w, adv_b = adv_clfLocals.w.get_data()[0], adv_clfLocals.b.get_data()[0]
        #     adv_m = np.sqrt(adv_w.dot(adv_w))
        #     adv_w = adv_w / adv_m
        #     adv_b /= adv_m
        #     all_updates.append(np.append(adv_w,adv_b))

        # iter = 40
        # for _ in range(iter):
        #     hSets = np.vstack([all_updates, agg_updates])

        #     krum_scores = []
        #     byzantine_client_num = 1
        #     # print(f'lambda {lambda_} number of adversary {advNumber} and updates {len(all_updates)} and hSets {len(hSets)}')
        #     for i in range(0, len(hSets)):
        #         dists = []
        #         for j in range(0, len(hSets)):
        #             if i != j:
        #                 temp_vector = np.subtract(hSets[i], hSets[j])
        #                 distance = np.dot(temp_vector,temp_vector)
        #                 dists.append(distance)
        #         #   print(i, "all_socre", dists)
        #         dists.sort()  # ascending
        #         score = dists[0 : len(hSets) - byzantine_client_num - 2]
        #         # print(i, "low_socre", score)
        #         krum_scores.append(sum(score))
        #         # print(i, krum_scores[-1])
        #     score_index = np.argsort(krum_scores)  # indices; ascending
        #     #   print(krum_scores)
        #     #   print(score_index)
        #     score_index = score_index[0 : len(hSets) - byzantine_client_num - 2]
        #     if advNumber == score_index[0]:
        #         break
        #     agg_updates = (agg_updates+orig_updates) / 2
        # tw = agg_updates[0: len(agg_updates) - 1]
        # b = agg_updates[len(agg_updates) - 1]
        # print(f'The iter = {_} b = {b}\n')

        old_w, old_b = tw,b
        return tw, b
    old_w, old_b = w, b
    return w, b



def load_local_data(advNumber, local_machine):
    global old_x_local, old_y_local
    # return XGlobal, yGlobal

    if local_machine>=advNumber: # if non attacker no dataset is changed
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])
        # print("labels of ", local_machine, ylocal)
        return Xlocal, ylocal


    if MODE=="COLLABORAT":
        if local_machine !=0:
            return old_x_local, old_y_local
        Xlocal = deepcopy(XGlobal[0 : advNumber * slc])
        ylocal = deepcopy(yGlobal[0 : advNumber * slc])
    else:
        Xlocal = deepcopy(XGlobal[local_machine * slc : (local_machine + 1) * slc])
        ylocal = deepcopy(yGlobal[local_machine * slc : (local_machine + 1) * slc])



    if ATTACK_TYPE == 'LABEL_FLIP':
        for j in range(len(ylocal)):
            ylocal[j] ^= 1
        old_x_local, old_y_local = Xlocal, ylocal
        return Xlocal, ylocal

    if ATTACK_TYPE == 'OPTIMAL':
        n_tr = int(len(ylocal)*0.4)  # Number of training set samples
        n_val = int(len(ylocal)*0.5)  # Number of validation set samples
        n_ts = len(ylocal) - n_tr - n_val  # Number of test set samples

        # Split in training, validation and test
        dataset = CDataset(Xlocal, ylocal)
        random_state = 999
        splitter = CTrainTestSplit(train_size=n_tr + n_val, test_size=n_ts, random_state=random_state)
        tr_val, ts = splitter.split(dataset)
        splitter = CTrainTestSplit(train_size=n_tr, test_size=n_val, random_state=random_state)
        tr, val = splitter.split(dataset)
        tr = tr.append(ts)

        clf = CClassifierSVM(kernel=CKernelLinear(), C=1)

        # We can now fit the classifier
        clf.fit(tr.X, tr.Y)

        # Should be chosen depending on the optimization problem
        lb, ub = 0., 1.  # Bounds of the attack space. Can be set to `None` for unbounded

        # Should be chosen depending on the optimization problem
        solver_params = {
            'eta': 0.25,
            'eta_min': 2.0,
            'eta_max': None,
            'max_iter': 200,
            'eps': 1e-8
        }

        pois_attack = CAttackPoisoningSVM(classifier=clf,
                                      training_data=tr,
                                      val=val,
                                      lb=lb, ub=ub,
                                      solver_params=solver_params,
                                      random_seed=random_state)

        n_poisoning_points = 20  # Number of poisoning points to generate
        pois_attack.n_points = n_poisoning_points

        #Run the poisoning attack
        # print("Attack started...")
        pois_y_pred, pois_scores, pois_ds, f_opt = pois_attack.run(ts.X, ts.Y)
        # print("Attack complete!")
        # tr = tr.append(pois_ds) #.append(val)
        Xlocal, ylocal = pois_ds.X.get_data(), pois_ds.Y.get_data()
        old_x_local, old_y_local = Xlocal, ylocal
        # print(Xlocal)
        # print(ylocal)
        return Xlocal, ylocal
    #if ATTACK_TYPE == 'RANDOM' or ATTACK_TYPE == 'OUR_ATTACK' or ATTACK_TYPE == 'OUR_ATTACK_GD':
    # if model attacker then no dataset is changed
    old_x_local, old_y_local = Xlocal, ylocal
    return Xlocal, ylocal






def dataLoad(p = 50):
    global dataset_type, XGlobal, yGlobal, testX, testy, slc
    if dataset_type == 'MNIST':
        # MNIST dataset will be downloaded and cached if needed
        loader = CDataLoaderMNIST()

        slc = 100
        size_all = 784

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples
        digits = (9, 8)

        dataset = loader.load('training', digits=digits)
        # Normalize the features in `[0, 1]`
        dataset.X /= 255
        XGlobal, yGlobal = dataset.X.get_data(), dataset.Y.get_data()

        test_dataset = loader.load('testing', digits=digits, num_samples=1000)
        test_dataset.X /= 255
        testX, testy = test_dataset.X.get_data(), test_dataset.Y.get_data()


    if dataset_type == 'IJCNN1':

        XGlobal, yGlobal = load_svmlight_file('/content/sample_data/ijcnn1.tr.bz2')
        XGlobal = XGlobal.toarray()
        yGlobal = [0 if y < 0 else 1 for y in yGlobal]

        testX, testy = load_svmlight_file('/content/sample_data/ijcnn1.t.bz2')
        testX = testX.toarray()
        testy = [0 if y < 0 else 1 for y in testy]

        slc = 300

    if dataset_type == 'FASHION_MNIST':
        # MNIST dataset will be downloaded and cached if needed
        train_df = pd.read_csv('/content/sample_data/fashion-mnist_train.csv')
        test_df = pd.read_csv('/content/sample_data/fashion-mnist_test.csv')
        print('The shape of training dataset : ', train_df.shape)
        print('The shape of testing dataset : ', test_df.shape)

        train = np.array(train_df, dtype = 'float32')
        test = np.array(test_df, dtype = 'float32')


        trainX = train[:,1:]/255

        trainY = train[:,0]

        dataX = test[:,1:]/255

        dataY=test[:,0]

        def gen_data(X, Y):
            retX, retY = [], []
            for i in range(len(Y)):
                # print(i, trainY[i])
                lbl = int(Y[i])
                if lbl == 0 or lbl == 6:
                    retX.append(X[i])
                    retY.append(0 if lbl == 0 else 1)
            # class_names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
            #    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
            # plt.figure(figsize=(17, 17))
            # for i in range(25):
            #     plt.subplot(5, 5, i + 1)
            #     plt.grid(False)
            #     plt.imshow(retX[i].reshape((28,28)))
            #     plt.colorbar()
            #     label_index = int(retY[i])
            #     plt.title(class_names[label_index])
            # plt.show()
            return retX, retY

        XGlobal, yGlobal = gen_data(trainX, trainY)
        unique, counts = np.unique(yGlobal, return_counts=True)
        print(f"Global count {dict(zip(unique, counts))}")
        testX, testy = gen_data(dataX, dataY)

        print(len(trainY), len(dataY))

        unique, counts = np.unique(testy, return_counts=True)
        print(f"Test count {dict(zip(unique, counts))}")
        print(len(yGlobal), testy)

        slc = 100
        size_all = 784
        # class_names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        #        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
        # plt.figure(figsize=(17, 17))

        # for i in range(25):
        #     plt.subplot(5, 5, i + 1)
        #     plt.grid(False)
        #     plt.imshow(testX[i].reshape((28,28)))
        #     plt.colorbar()
        #     label_index = int(testy[i])
        #     plt.title(class_names[label_index])
        # plt.show()

    elif dataset_type == 'CIFAR10':
        # CIFAR10 dataset will be downloaded and cached if needed
        loader = CDataLoaderCIFAR10()

        def rgb_2_gray(arr):
            return [0.2989 * arr[i] + 0.5870 * arr[i+1024] + 0.1140 * arr[i+2048] for i in range(1024)]

        size_all = 1024
        slc = 100

        # total MNIST data with 5 and 9 is 11370
        n_samples = slc * num_local_machine  # Number of samples

        dataset, test_dataset = loader.load()
        # Normalize the features in `[0, 1]`
        dataset.X /= 255

        XGlobal, yGlobal = [], []
        trainX = dataset.X.get_data()
        trainY = dataset.Y.get_data()

        for i in range(len(trainY)):
            if trainY[i] < 2:
                XGlobal.append(rgb_2_gray(trainX[i]))
                yGlobal.append(trainY[i])

        unique, counts = np.unique(yGlobal, return_counts=True)
        print(f"Global count {dict(zip(unique, counts))}")
        testX, testy = [], []
        test_dataset.X /= 255
        dataX = test_dataset.X.get_data()
        dataY = test_dataset.Y.get_data()

        print(len(trainY), len(dataY))

        for i in range(len(dataY)):
            if dataY[i] < 2:
                testX.append(rgb_2_gray(dataX[i]))
                testy.append(dataY[i])

        unique, counts = np.unique(testy, return_counts=True)
        print(f"Test count {dict(zip(unique, counts))}")
        print(len(yGlobal), len(testy))

    if p == -1:
        data = [[], []]
        sz = len(yGlobal)
        for i in range(sz):
            data[yGlobal[i]].append(XGlobal[i])
        XGlobal, yGlobal = [], []
        print(f"Changing data, p={p} and slc = {slc}")
        for edge_device in range(num_local_machine):
            label = 0
            data_iid = edge_device + (0 if edge_device >= (num_local_machine//2) else 1)
            print(f"edge_device {edge_device}, 0: {len(data[0])}, 1: {len(data[1])}, data_iid {data_iid}")
            for _ in range(data_iid):
                XGlobal.append(data[label].pop())
                yGlobal.append(label)

            anti_label = 1
            for _ in range(slc - data_iid):
                XGlobal.append(data[anti_label].pop())
                yGlobal.append(anti_label)
    elif p != 50:
        data = [[], []]
        sz = len(yGlobal)
        for i in range(sz):
            data[yGlobal[i]].append(XGlobal[i])
        XGlobal, yGlobal = [], []
        data_iid = int((slc * p) / 100 + 0.5)
        print(f"Changing data, p={p} and data_iid = {data_iid} and slc = {slc}")
        for edge_device in range(num_local_machine):
            label = edge_device%2
            for _ in range(data_iid):
                XGlobal.append(data[label].pop())
                yGlobal.append(label)

            anti_label = ( edge_device+1 ) % 2
            for _ in range(slc - data_iid):
                XGlobal.append(data[anti_label].pop())
                yGlobal.append(anti_label)



In [ ]:
def outlier_detection(hSets, advNumber, numberMachine):
    # detection using KNN

    colors = []
    for _ in range(advNumber):
        colors.append('r')

    for _ in range(numberMachine-advNumber):
        colors.append('b')

    print(f'kNN based out detection')
    outlier_clf = KNN(contamination=0.2)
    outlier_clf.fit(hSets)

    y_train_pred = outlier_clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = outlier_clf.decision_scores_  # raw outlier scores
    # for i in range(len(hSets)):
    #     print("Machine {}, prediction {} outlier score {}".format(i, y_train_pred[i], y_train_scores[i]))
    print(f"For kNN with adversary {advNumber}, adversarial : min: {np.amin(y_train_scores[0:advNumber])} max: {np.amax(y_train_scores[0:advNumber])} avg: {np.mean(y_train_scores[0:advNumber])}")
    print(f"For kNN with adversary {advNumber}, non adversarial : min: {np.amin(y_train_scores[advNumber:])} max: {np.amax(y_train_scores[advNumber:])} avg: {np.mean(y_train_scores[advNumber:])}")
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # ax.bar(numberMachine,y_train_scores,color=colors)
    # ax.set_xlabel('Local Machine')
    # ax.set_ylabel('Outlier Score')
    # plt.xlabel('Local Machine', fontsize=20)
    # plt.ylabel('Outlier Score', fontsize=20)
    # plt.xticks(fontsize=16)
    # plt.yticks(fontsize=16)
    # plt.show()



    # detection using k means clustering

    # print('k means clustering based out detection')
    # outlier_clf_kmeans = CBLOF(beta=len(hSets))
    # outlier_clf_kmeans.fit(hSets)
    # y_train_pred = outlier_clf_kmeans.labels_  # binary labels (0: inliers, 1: outliers)
    # y_train_scores = outlier_clf_kmeans.decision_scores_  # raw outlier scores
    # for i in range(len(hSets)):
    #         print("Machine {}, prediction {} outlier score {}".format(i, y_train_pred[i], y_train_scores[i]))
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # ax.bar(numberMachine,y_train_scores,color=colors)
    # ax.set_xlabel('Local Machine')
    # ax.set_ylabel('Outlier Score')
    # plt.xlabel('Local Machine', fontsize=20)
    # plt.ylabel('Outlier Score', fontsize=20)
    # plt.xticks(fontsize=16)
    # plt.yticks(fontsize=16)
    # plt.show()


    # detection using Histogram- based outlier detection

    print('Histogram based out detection')
    outlier_clf_hbos = HBOS(n_bins=3)
    outlier_clf_hbos.fit(hSets)
    y_train_pred = outlier_clf_hbos.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = outlier_clf_hbos.decision_scores_  # raw outlier scores
    # for i in range(len(hSets)):
    #         print("Machine {}, prediction {} outlier score {}".format(i, y_train_pred[i], y_train_scores[i]))
    print(f"For Histogram with adversary {advNumber}, adversarial : min: {np.amin(y_train_scores[0:advNumber])} max: {np.amax(y_train_scores[0:advNumber])} avg: {np.mean(y_train_scores[0:advNumber])}")
    print(f"For Histogram with adversary {advNumber}, non adversarial : min: {np.amin(y_train_scores[advNumber:])} max: {np.amax(y_train_scores[advNumber:])} avg: {np.mean(y_train_scores[advNumber:])}")
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # ax.bar(numberMachine,y_train_scores,color=colors)
    # ax.set_xlabel('Local Machine')
    # ax.set_ylabel('Outlier Score')
    # plt.xlabel('Local Machine', fontsize=20)
    # plt.ylabel('Outlier Score', fontsize=20)
    # plt.xticks(fontsize=16)
    # plt.yticks(fontsize=16)
    # plt.show()


    # detection using Copula Based Outlier Detector

    print('Copula Based Outlier Detector')
    outlier_clf_copod = COPOD()
    outlier_clf_copod.fit(hSets)
    y_train_pred = outlier_clf_copod.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = outlier_clf_copod.decision_scores_  # raw outlier scores
    # for i in range(len(hSets)):
    #         print("Machine {}, prediction {} outlier score {}".format(i, y_train_pred[i], y_train_scores[i]))
    print(f"For Copula with adversary {advNumber}, adversarial : min: {np.amin(y_train_scores[0:advNumber])} max: {np.amax(y_train_scores[0:advNumber])} avg: {np.mean(y_train_scores[0:advNumber])}")
    print(f"For Copula with adversary {advNumber}, non adversarial : min: {np.amin(y_train_scores[advNumber:])} max: {np.amax(y_train_scores[advNumber:])} avg: {np.mean(y_train_scores[advNumber:])}")
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # ax.bar(numberMachine,y_train_scores,color=colors)
    # ax.set_xlabel('Local Machine')
    # ax.set_ylabel('Outlier Score')
    # plt.xlabel('Local Machine', fontsize=20)
    # plt.ylabel('Outlier Score', fontsize=20)
    # plt.xticks(fontsize=16)
    # plt.yticks(fontsize=16)
    # plt.show()


In [ ]:
#@title Default title text
dataset_type = 'IJCNN1' # 'MNIST' # 'CIFAR10' # 'FASHION_MNIST' #
ATTACK_TYPE = 'OPTIMAL' # 'OUR_ATTACK_GD' # 'OUR_ATTACK' #  'FANG_KRUM' # 'AGR_KRUM' # 'LABEL_FLIP' # 'RANDOM' # 'LIE' #'OUR_ATTACK_BND_KRUM' # 'GRADIENT' #
MODE = 'INDIV' # 'COLLABORAT' #   #
num_local_machine = 100
iid_deg=50

for dataset_type, ATTACK_TYPE, iid_deg in itertools.product(['CIFAR10', 'FASHION_MNIST', 'IJCNN1'], ['LABEL_FLIP', 'OPTIMAL'], range(50, 51, 10)) :
# for iid_deg in range(50, 51, 10):
    # print(dataset_type, ATTACK_TYPE, MODE, num_local_machine)
    print()
    print(dataset_type, ATTACK_TYPE, MODE, num_local_machine, iid_deg)
    dataLoad(iid_deg)


    # for advNumber in [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]: # advNumber means how many attackers
    for advNumber in [30]: # advNumber means how many attackers

        hSets = []
        for local_machine in range(num_local_machine):
            X_local, y_local= load_local_data(advNumber, local_machine)

            # print(local_machine, y_local)

            clfLocals = CClassifierSVM(C=1)
            clfLocals.fit(X_local, y_local)

            w, b = clfLocals.w.get_data()[0], clfLocals.b.get_data()[0]
            m = np.sqrt(w.dot(w))
            w = w / m
            b /= m

            w, b = model_change(advNumber, local_machine, w, b)
            hSets.append(np.append(w,b))


        outler_input = deepcopy(hSets)
        if advNumber == 30:
            outlier_detection(deepcopy(hSets), advNumber, num_local_machine)
        # Average
        w = np.mean(hSets, axis=0)

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after average')

            # First
        w = hSets[0]

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker poisonus hyperplane')


            # Average
        w = hSets[70]

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker good hyperplane')



        # Clustering
        kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
        w = kmeans.cluster_centers_[0]

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
            # print(det, testX[i], testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after clustering')


        # Convergence
        isUpdated = True
        iter = 10000
        while isUpdated and iter > 0:
            iter -= 1
            isUpdated = False
            kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
            w = kmeans.cluster_centers_[0]
            for i in range(num_local_machine):
                if not np.allclose(w, hSets[i]):
                    hSets[i] = np.mean( np.array([ hSets[i], w ]), axis=0 )
                    isUpdated = True

        kmeans = KMeans(n_clusters=1, random_state=0).fit(hSets)
        w = kmeans.cluster_centers_[0]
        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1): cnt += 1
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after convergence')

            # Krum
        hSets = deepcopy(outler_input)

        krum_scores = []
        byzantine_client_num = num_local_machine // 5
        for i in range(0, num_local_machine):
            dists = []
            for j in range(0, num_local_machine):
                if i != j:
                    temp_vector = np.subtract(hSets[i], hSets[j])
                    distance = np.dot(temp_vector,temp_vector)
                    dists.append(distance)
            #   print(i, "all_socre", dists)
            dists.sort()  # ascending
            score = dists[0 : num_local_machine - byzantine_client_num - 2]
            #   print(i, "low_socre", score)
            krum_scores.append(sum(score))
            #   print(i, sum(score))
        score_index = np.argsort(krum_scores)  # indices; ascending
        score_index = score_index[0 : num_local_machine - byzantine_client_num - 2]
        #   print(krum_scores)
        #   print(score_index)
        hSets = [outler_input[i] for i in score_index]

        w = hSets[0]

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after krum')

        w = np.mean(hSets, axis=0)

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after multi-krum')



        # Trimmed Mean
        hSets = deepcopy(outler_input)

        trimmed_num = int(len(hSets)*0.2)
        temp_hSets = np.zeros((len(hSets) - 2*trimmed_num, len(hSets[0])))
        #   print(temp_hSets)
        for j in range(len(hSets[0])):
            temp_hSet = []
            for i in range(len(hSets)):
                temp_hSet.append(hSets[i][j])
            temp_hSet.sort()
            #   print(temp_hSet)
            for i in range(trimmed_num, len(temp_hSet) - trimmed_num):
                #   print(i, j, i-trimmed_num)
                temp_hSets[i-trimmed_num][j] = temp_hSet[i]
        hSets = temp_hSets

        w = np.mean(hSets, axis=0)

        cnt = 0
        for i in range(len(testX)):
            det = np.dot(np.append(testX[i], 1), w)
            if (det < 0 and testy[i] == 0) or (det > 0 and testy[i] == 1):
                cnt += 1
            # print(i, det, testy[i])
        print(f'Time {datetime.now()}: Accuracy is {cnt}/{len(testX)} = {cnt/len(testX)} with {advNumber} attacker after trimmed mean')


CIFAR10 LABEL_FLIP INDIV 100 50
Global count {0: 5000, 1: 5000}
50000 10000
Test count {0: 1000, 1: 1000}
10000 2000
kNN based out detection
For kNN with adversary 30, adversarial : min: 1.3065728925728082 max: 1.391500524514079 avg: 1.3428413295486408
For kNN with adversary 30, non adversarial : min: 1.2520679245513524 max: 1.3719337301788113 avg: 1.3210115750051385
Histogram based out detection
For Histogram with adversary 30, adversarial : min: -3055.860482026873 max: -2967.5147036330363 avg: -3006.9136083566777
For Histogram with adversary 30, non adversarial : min: -3118.2636929027976 max: -2995.9056230239157 avg: -3061.2452043460107
Copula Based Outlier Detector
For Copula with adversary 30, adversarial : min: 1375.5742147342676 max: 1452.049099412915 avg: 1414.5808466856927
For Copula with adversary 30, non adversarial : min: 1250.7272340855125 max: 1363.6201303592156 avg: 1299.8053937916418
Time 2023-08-11 15:14:07.611447: Accuracy is 1505/2000 = 0.7525 with 30 attacker after 

ValueError: ignored

In [ ]:
gd_hSets = deepcopy(outler_input)

In [ ]:
bnd_hSets = deepcopy(outler_input)

In [ ]:
temp_vector = np.subtract([1,2,3], [5,6,3])
distance = np.dot(temp_vector,temp_vector)
print(distance)

32


In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('/content/sample_data/fashion-mnist_train.csv')
test_df = pd.read_csv('/content/sample_data/fashion-mnist_test.csv')
print('The shape of training dataset : ', train_df.shape)
print('The shape of testing dataset : ', test_df.shape)

train = np.array(train_df, dtype = 'float32')
test = np.array(test_df, dtype = 'float32')


x_train = train[:,1:]/255

y_train = train[:,0]

x_test= test[:,1:]/255

y_test=test[:,0]


X_train, X_validate,y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.2, random_state = 5000)
print('The size of training data after model selection : ', X_train.shape, y_train.shape)
print('The size of Validation data after model selection : ', X_validate.shape, y_validate.shape)

The shape of training dataset :  (60000, 785)
The shape of testing dataset :  (10000, 785)
The size of training data after model selection :  (48000, 784) (48000,)
The size of Validation data after model selection :  (12000, 784) (12000,)


In [ ]:
import bz2
from sklearn.datasets import load_svmlight_file
import pandas as pd

# Open the compressed file
# Load the LibSVM format data
X, y = load_svmlight_file('/content/sample_data/ijcnn1.t.bz2')
print(len(X.toarray()[0]), len(y))
print(np.unique(y, return_counts=True))

# Convert to a dense array and then to a DataFrame
df = pd.DataFrame(X.toarray())

# Adding target variable to DataFrame
df['target'] = y

# Display the first few rows of the DataFrame
print(df.head())


22 91701
(array([-1.,  1.]), array([82989,  8712]))
     0    1    2    3    4    5    6    7    8    9  ...        13        14  \
0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.000292 -0.020372   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ... -0.020372  0.007305   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.007305  0.002519   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.002519  0.018198   
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.018198 -0.024526   

         15        16        17        18        19        20        21  \
0  0.007305  0.002519  0.018198 -0.024526  0.012458 -0.016884  0.006842   
1  0.002519  0.018198 -0.024526  0.012458 -0.016884  0.006842  0.005018   
2  0.018198 -0.024526  0.012458 -0.016884  0.006842  0.005018  0.016210   
3 -0.024526  0.012458 -0.016884  0.006842  0.005018  0.016210 -0.027162   
4  0.012458 -0.016884  0.006842  0.005018  0.016210 -0.027162  0.026788   

   target  
0   

In [ ]:
import os
print(os.getcwd())
!rm /content/sample_data/fashion-mnist_train.csv
!rm /content/sample_data/fashion-mnist_test.csv
!wget https://dax-cdn.cdn.appdomain.cloud/dax-fashion-mnist/1.0.2/fashion-mnist.tar.gz -O /content/sample_data/fashion-mnist.tar.gz
!tar -xvzf /content/sample_data/fashion-mnist.tar.gz -C /content/sample_data/
!ls -lS /content/sample_data/
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content
rm: cannot remove '/content/sample_data/fashion-mnist_train.csv': No such file or directory
rm: cannot remove '/content/sample_data/fashion-mnist_test.csv': No such file or directory
--2023-07-20 09:21:43--  https://dax-cdn.cdn.appdomain.cloud/dax-fashion-mnist/1.0.2/fashion-mnist.tar.gz
Resolving dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)... 23.54.209.135, 2600:1408:20:599::d2d, 2600:1408:20:587::d2d
Connecting to dax-cdn.cdn.appdomain.cloud (dax-cdn.cdn.appdomain.cloud)|23.54.209.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38564215 (37M) [application/x-tar]
Saving to: ‘/content/sample_data/fashion-mnist.tar.gz’

/content/sample_dat 100%[===================>]  36.78M  72.4MB/s    in 0.5s    

2023-07-20 09:21:44 (72.4 MB/s) - ‘/content/sample_data/fashion-mnist.tar.gz’ saved [38564215/38564215]

fashion-mnist_test.csv
fashion-mnist_train.csv
LICENSE.txt
total 248668
-rw-r--r-- 1 root root 133047193 Dec  6  2017 fashion-mnist_tr

In [ ]:
import os
print('Loading ijcnn1 dataset')
print(os.getcwd())
!rm /content/sample_data/ijcnn1.t.bz2
!rm /content/sample_data/ijcnn1.tr.bz2
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/ijcnn1.t.bz2 -O /content/sample_data/ijcnn1.t.bz2
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/ijcnn1.tr.bz2 -O /content/sample_data/ijcnn1.tr.bz2
!wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/ijcnn1.val.bz2 -O /content/sample_data/ijcnn1.val.bz2
!ls -lS /content/sample_data/
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Loading ijcnn1 dataset
/content
rm: cannot remove '/content/sample_data/ijcnn1.t.bz2': No such file or directory
rm: cannot remove '/content/sample_data/ijcnn1.tr.bz2': No such file or directory
--2023-07-20 09:21:19--  https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/ijcnn1.t.bz2
Resolving www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)... 140.112.30.26
Connecting to www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)|140.112.30.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2597597 (2.5M) [application/x-bzip2]
Saving to: ‘/content/sample_data/ijcnn1.t.bz2’

/content/sample_dat 100%[===================>]   2.48M  1.84MB/s    in 1.3s    

2023-07-20 09:21:22 (1.84 MB/s) - ‘/content/sample_data/ijcnn1.t.bz2’ saved [2597597/2597597]

--2023-07-20 09:21:22--  https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/ijcnn1.tr.bz2
Resolving www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)... 140.112.30.26
Connecting to www.csie.ntu.edu.tw (www.csie.ntu.edu.tw)|140